# NLP basic challenges

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

## Word Frequency Counter 📊 Challenge:
Write a Python function that takes a text document as input and returns a dictionary containing the frequency of each word in the document.

### 1-basic approach

In [5]:
import re

def count_doc(doc_path):
    vocab = {}
    with open(doc_path, 'r', encoding='utf-8') as doc:
        for line in doc:
            # Remove punctuation and convert to lowercase
            words = re.findall(r'\b\w+\b', line.lower())
            for word in words:
                if word not in vocab:
                    vocab[word] = 1
                else:
                    vocab[word] += 1
    return vocab


In [6]:
count_doc("test.txt")

{'the': 7,
 'lost': 1,
 'key': 4,
 'once': 1,
 'upon': 1,
 'a': 2,
 'time': 1,
 'anna': 2,
 'misplaced': 1,
 'her': 4,
 'she': 5,
 'searched': 1,
 'under': 1,
 'couch': 1,
 'inside': 1,
 'bag': 1,
 'and': 2,
 'even': 1,
 'in': 1,
 'fridge': 1,
 'but': 1,
 'it': 1,
 'was': 1,
 'nowhere': 1,
 'to': 3,
 'be': 1,
 'found': 2,
 'frustrated': 1,
 'sat': 1,
 'down': 1,
 'think': 1,
 'suddenly': 1,
 'remembered': 1,
 'playing': 1,
 'with': 1,
 'cat': 2,
 'whiskers': 3,
 'earlier': 1,
 'that': 1,
 'morning': 1,
 'could': 1,
 'have': 1,
 'taken': 1,
 'followed': 1,
 'living': 1,
 'room': 1,
 'where': 1,
 'hidden': 1,
 'beneath': 1,
 'pile': 1,
 'of': 1,
 'cushions': 1,
 'relieved': 1,
 'laughed': 1,
 'thanked': 1,
 'for': 1,
 'unexpected': 1,
 'adventure': 1}

### 2- map-reduce approach

### **Concept de MapReduce :**
MapReduce est un paradigme de programmation utilisé pour traiter de grandes quantités de données de manière distribuée. Il se divise en deux étapes principales :

1. **Map** : 
   - Divise le travail en tâches indépendantes.
   - Transforme les données d'entrée en paires clé-valeur `(clé, valeur)`.
   - Exemples : Compter les mots, générer `(mot, 1)` pour chaque mot trouvé.

2. **Reduce** :
   - Regroupe les paires ayant la même clé.
   - Applique une opération (comme une somme, une moyenne, etc.) pour réduire plusieurs valeurs associées à une clé en une seule.
   - Exemples : Additionner les valeurs pour obtenir la fréquence totale de chaque mot.

---

### **Étapes de MapReduce :**
1. **Split** :
   - Diviser les données d'entrée en morceaux (partitions).
   - Exemple : Un fichier texte de 1 To divisé en blocs de 64 Mo.

2. **Map** :
   - Traiter chaque morceau de manière indépendante pour générer des paires clé-valeur.
   - Exemple : Depuis un texte, produire `(mot, 1)` pour chaque mot.

3. **Shuffle and Sort** :
   - Regrouper et trier les données par clé (toutes les occurrences de la même clé ensemble).
   - Exemple : Regrouper tous les `(mot, 1)` par mot.

4. **Reduce** :
   - Effectuer une opération sur les groupes de clés triés.
   - Exemple : Calculer la somme des valeurs pour chaque clé `(mot, total)`.

5. **Output** :
   - Produire les résultats finaux.
   - Exemple : Une liste contenant chaque mot et sa fréquence.

---

### **Illustration avec un exemple (comptage de mots) :**
#### Entrée :
```
Texte : "Bonjour monde, bonjour Python"
```

#### Étapes :
1. **Map** :
   ```
   Bonjour -> (bonjour, 1)
   monde -> (monde, 1)
   bonjour -> (bonjour, 1)
   Python -> (python, 1)
   ```

2. **Shuffle and Sort** :
   ```
   (bonjour, [1, 1]), (monde, [1]), (python, [1])
   ```

3. **Reduce** :
   ```
   (bonjour, 2), (monde, 1), (python, 1)
   ```

#### Résultat final :
```
{'bonjour': 2, 'monde': 1, 'python': 1}
```

---

### **Avantages :**
- **Scalabilité** : Idéal pour traiter des données massives sur plusieurs machines.
- **Parallélisme** : Les étapes Map et Reduce peuvent être parallélisées.

### **Limitation** :
- Pas adapté à tous les problèmes (par exemple, ceux nécessitant de fortes dépendances entre données).

In [10]:
import re

def map_words(doc_path):
    with open(doc_path, 'r', encoding='utf-8') as doc:
        for line in doc:
            words = re.findall(r'\b\w+\b', line.lower())  # Clean and tokenize
            for word in words:
                yield (word, 1)  # Yield each word with a count of 1

# Reduce function: aggregates counts for each word
def reduce_word_counts(mapped_words):
    word_counts = {}
    for word, count in mapped_words:
        if word not in word_counts:
            word_counts[word] = 1  # Sum up counts for each word
        else:
            word_counts[word] += count
    return word_counts


# Combine Map and Reduce
def map_reduce_word_count(doc_path):
    mapped_words = map_words(doc_path)  # Step 1: Map
    reduced_counts = reduce_word_counts(mapped_words)  # Step 2: Reduce
    return reduced_counts

In [11]:
map_reduce_word_count("test.txt")

{'the': 7,
 'lost': 1,
 'key': 4,
 'once': 1,
 'upon': 1,
 'a': 2,
 'time': 1,
 'anna': 2,
 'misplaced': 1,
 'her': 4,
 'she': 5,
 'searched': 1,
 'under': 1,
 'couch': 1,
 'inside': 1,
 'bag': 1,
 'and': 2,
 'even': 1,
 'in': 1,
 'fridge': 1,
 'but': 1,
 'it': 1,
 'was': 1,
 'nowhere': 1,
 'to': 3,
 'be': 1,
 'found': 2,
 'frustrated': 1,
 'sat': 1,
 'down': 1,
 'think': 1,
 'suddenly': 1,
 'remembered': 1,
 'playing': 1,
 'with': 1,
 'cat': 2,
 'whiskers': 3,
 'earlier': 1,
 'that': 1,
 'morning': 1,
 'could': 1,
 'have': 1,
 'taken': 1,
 'followed': 1,
 'living': 1,
 'room': 1,
 'where': 1,
 'hidden': 1,
 'beneath': 1,
 'pile': 1,
 'of': 1,
 'cushions': 1,
 'relieved': 1,
 'laughed': 1,
 'thanked': 1,
 'for': 1,
 'unexpected': 1,
 'adventure': 1}

## Text Cleaning and Tokenization 🧹 Challenge:

Create a function to clean and tokenize a given text, removing punctuation and converting words to lowercase

In [12]:
test = "hello, world"
test.split()

['hello,', 'world']

In [13]:
def clean_text(text):
    # lower the text
    text = text.lower()
    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # tokenize
    return text.split()

In [14]:
test = "Once upon a time, Anna misplaced her key. She searched under the couch, inside her bag,"
clean_text(test)

['once',
 'upon',
 'a',
 'time',
 'anna',
 'misplaced',
 'her',
 'key',
 'she',
 'searched',
 'under',
 'the',
 'couch',
 'inside',
 'her',
 'bag']

> re.sub(r'[^\w\s]', '', text) :

Supprime tous les caractères qui ne sont ni alphanumériques (\w) ni des espaces (\s).

> re.sub(r'\s+', ' ', text).strip() :

Remplace plusieurs espaces par un seul, puis enlève les espaces superflus au début ou à la fin.

## Stopword Removal 🚫 Challenge

Develop a function that removes stopwords (commonly used words) from a given sentence or text.

In [15]:
import nltk
from nltk.corpus import stopwords

# Téléchargement des stop words (à faire une seule fois)
nltk.download('stopwords')

def remove_stop_words(text):
    # Définir la liste des stop words en anglais
    stop_words = set(stopwords.words('english'))
    # Supprimer les stop words du texte
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tariq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
test = "Once upon a time, Anna misplaced her key. She searched under the couch, inside her bag,"
remove_stop_words(test)

'Once upon time, Anna misplaced key. She searched couch, inside bag,'

## Sentiment Analysis 🌟 Challenge:

Voici les étapes principales du processus **TF-IDF (Term Frequency-Inverse Document Frequency)** :

1. **Calcul du Term Frequency (TF)** :
   - Le **TF** mesure la fréquence d'apparition d'un mot dans un document par rapport à l'ensemble du document. 
   - Formellement : 
     \[$
     TF(t, d) = \frac{\text{Nombre d'occurrences du terme } t \text{ dans le document } d}{\text{Nombre total de termes dans le document } d}
     $\]

2. **Calcul de l'Inverse Document Frequency (IDF)** :
   - L'**IDF** mesure l'importance d'un terme dans l'ensemble des documents. Plus un mot apparaît dans peu de documents, plus il est important.
   - Formellement :
     \[$
     IDF(t) = \log\left(\frac{\text{Nombre total de documents}}{\text{Nombre de documents contenant le terme } t}\right)
     $\]

3. **Calcul du TF-IDF** :
   - Le **TF-IDF** est le produit du **TF** et du **IDF**, ce qui permet de mesurer l'importance d'un terme dans un document par rapport à l'ensemble du corpus.
   - Formellement :
     \[$
     TFIDF(t, d) = TF(t, d) \times IDF(t)
     $\]

---

### **Résumé du processus** :
1. Calculez le **TF** pour chaque mot dans chaque document.
2. Calculez le **IDF** pour chaque mot dans l'ensemble des documents.
3. Multipliez les valeurs **TF** et **IDF** pour chaque mot dans chaque document pour obtenir le **TF-IDF**.

Le **TF-IDF** donne plus de poids aux mots fréquents dans un document, mais rares dans l'ensemble du corpus, ce qui permet de capturer les termes significatifs.

In [17]:
import nltk
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Download required NLTK datasets
nltk.download('movie_reviews')
nltk.download('stopwords')

# Load the dataset
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

# Shuffle the dataset
import random
random.shuffle(documents)

# Preprocess text (remove stop words and convert to lowercase)
stop_words = set(stopwords.words('english'))

def preprocess(words):
    return ' '.join([word.lower() for word in words if word.lower() not in stop_words])

# Prepare the data
texts = [preprocess(words) for words, label in documents]
labels = [label for _, label in documents]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=42)

# Convert text data into numerical features using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to top 5000 features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a classifier (e.g., Naive Bayes)
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = classifier.predict(X_test_tfidf)

# Evaluate the model
print(classification_report(y_test, y_pred))


[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\tariq\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tariq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              precision    recall  f1-score   support

         neg       0.73      0.87      0.79       288
         pos       0.86      0.70      0.77       312

    accuracy                           0.78       600
   macro avg       0.79      0.79      0.78       600
weighted avg       0.80      0.78      0.78       600



In [18]:
# Test new reviews
def test_classifier(new_reviews):
    # Step 1: Preprocess the input reviews
    preprocessed_reviews = [preprocess(review.split()) for review in new_reviews]
    
    # Step 2: Transform the reviews into TF-IDF features
    tfidf_reviews = vectorizer.transform(preprocessed_reviews)
    
    # Step 3: Predict the sentiment
    predictions = classifier.predict(tfidf_reviews)
    
    # Step 4: Output results
    for review, sentiment in zip(new_reviews, predictions):
        print(f"Review: {review}")
        print(f"Predicted Sentiment: {sentiment}")
        print("-" * 30)

# Example new reviews
new_reviews = [
    "This movie was absolutely fantastic! I loved it.",
    "The plot was boring and the acting was terrible.",
    "It was okay, not the best but not the worst either."
]

# Test the classifier
test_classifier(new_reviews)


Review: This movie was absolutely fantastic! I loved it.
Predicted Sentiment: pos
------------------------------
Review: The plot was boring and the acting was terrible.
Predicted Sentiment: neg
------------------------------
Review: It was okay, not the best but not the worst either.
Predicted Sentiment: neg
------------------------------


### ATTENTION: 
>> TF-IDF est une méthode efficace pour transformer un document en vecteur, mais ce n'est pas un modèle embedding car il ne capture pas les >> relations contextuelles ou sémantiques entre les mots comme le font Word2Vec, GloVe, ou BERT.

## Named Entity Recognition (NER) 🧐 Challenge:
Implement a Named Entity Recognition algorithm that identifies and classifies named entities like names, locations, and organizations in a text.

In [23]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -- ------------------------------------- 0.8/12.8 MB 4.2 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 5.0 MB/s eta 0:00:03
     --------- ------------------------------ 3.1/12.8 MB 5.1 MB/s eta 0:00:02
     ------------- -------------------------- 4.2/12.8 MB 5.2 MB/s eta 0:00:02
     ---------------- ----------------------- 5.2/12.8 MB 5.1 MB/s eta 0:00:02
     ------------------- -------------------- 6.3/12.8 MB 5.1 MB/s eta 0:00:02
     ---------------------- ----------------- 7.3/12.8 MB 5.1 MB/s eta 0:00:02
     --------------------------- ------------ 8.7/12.8 MB 5.2 MB/s eta 0:00:01
     ------------------------------- -------- 10.0/12.8 MB 5.4 MB/s eta 0:00:01
     --------------------------------- ------ 10.7/12.8 MB 5.3 MB/s eta 0:00:01
     ------------------------------------ --- 11.5/12.8 MB 5.1 MB/s eta 0:00:01
     ---------------------------------------  12.6/12.8 

In [24]:
import spacy

def named_entity_recognition(text):
    """
    Identifie et classe les entités nommées dans un texte.
    
    Args:
        text (str): Le texte d'entrée pour l'analyse NER.
        
    Returns:
        dict: Un dictionnaire avec des catégories d'entités comme clés
              et les entités correspondantes comme valeurs (listes).
    """
    # Charger un modèle pré-entraîné de spaCy
    nlp = spacy.load("en_core_web_sm")  # Petit modèle anglais

    # Analyser le texte
    doc = nlp(text)

    # Initialiser un dictionnaire pour stocker les entités
    entities = {}

    # Parcourir les entités identifiées
    for ent in doc.ents:
        if ent.label_ not in entities:
            entities[ent.label_] = []
        entities[ent.label_].append(ent.text)

    return entities

# Exemple d'utilisation
if __name__ == "__main__":
    text = """Apple Inc., based in Cupertino, California, was founded by Steve Jobs, 
    Steve Wozniak, and Ronald Wayne in 1976. It is one of the largest technology 
    companies in the world."""
    
    result = named_entity_recognition(text)
    for entity_type, entity_list in result.items():
        print(f"{entity_type}: {entity_list}")


ORG: ['Apple Inc.']
GPE: ['Cupertino', 'California']
PERSON: ['Steve Jobs', 'Steve Wozniak', 'Ronald Wayne']
DATE: ['1976']


## Text Similarity 📜 Challenge: 
Create a function that measures the similarity between two text documents using techniques like TF-IDF or cosine similarity.

> - Proche de 1 : Les objets sont très similaires.
> - Proche de 0 : Les objets ne sont pas similaires.
> - Proche de -1 : Les objets sont opposés.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Function to clean the text (optional step to remove special characters)
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Function to compute similarity between two documents
def compute_similarity(doc1, doc2):
    # Clean the documents (optional)
    doc1 = clean_text(doc1)
    doc2 = clean_text(doc2)
    
    # Initialize TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    
    # Fit and transform the documents
    tfidf_matrix = tfidf_vectorizer.fit_transform([doc1, doc2])
    
    # Compute cosine similarity between the two document vectors
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    
    return cosine_sim[0][0]  # Cosine similarity score between 0 and 1

# Example usage
doc1 = "I love programming in Python. It's a great language for data science."
doc2 = "Python is awesome for data science and machine learning tasks."

similarity = compute_similarity(doc1, doc2)
print(f"Cosine Similarity: {similarity}")


Cosine Similarity: 0.25233420143369617


In [29]:
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean, cityblock

# Function to clean the text (optional step to remove special characters)
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Function to compute Cosine Similarity
def compute_cosine_similarity(doc1, doc2):
    # Initialize TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    
    # Fit and transform the documents
    tfidf_matrix = tfidf_vectorizer.fit_transform([doc1, doc2])
    
    # Compute cosine similarity between the two document vectors
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    
    return cosine_sim[0][0]  # Cosine similarity score between 0 and 1

# Function to compute Jaccard Similarity
def compute_jaccard_similarity(doc1, doc2):
    # Tokenize and get unique words
    words_doc1 = set(clean_text(doc1).split())
    words_doc2 = set(clean_text(doc2).split())
    
    # Compute Jaccard Similarity
    intersection = len(words_doc1.intersection(words_doc2))
    union = len(words_doc1.union(words_doc2))
    
    return intersection / union  # Jaccard similarity between 0 and 1

# Function to compute Euclidean Distance
def compute_euclidean_distance(doc1, doc2):
    # Initialize TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    
    # Fit and transform the documents
    tfidf_matrix = tfidf_vectorizer.fit_transform([doc1, doc2])
    
    # Compute Euclidean distance between the two document vectors (flatten to 1D)
    euclidean_dist = euclidean(tfidf_matrix[0].toarray().flatten(), tfidf_matrix[1].toarray().flatten())
    
    return euclidean_dist


# Function to compute Manhattan Distance
def compute_manhattan_distance(doc1, doc2):
    # Initialize TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    
    # Fit and transform the documents
    tfidf_matrix = tfidf_vectorizer.fit_transform([doc1, doc2])
    
    # Compute Manhattan (L1) distance between the two document vectors
    manhattan_dist = cityblock(tfidf_matrix[0].toarray().flatten(), tfidf_matrix[1].toarray().flatten())
    
    return manhattan_dist

# Example usage
doc1 = "I love programming in Python. It's a great language for data science."
doc2 = "Python is awesome for data science and machine learning tasks."

# Clean and compute different metrics
cosine_sim = compute_cosine_similarity(doc1, doc2)
jaccard_sim = compute_jaccard_similarity(doc1, doc2)
euclidean_dist = compute_euclidean_distance(doc1, doc2)
manhattan_dist = compute_manhattan_distance(doc1, doc2)

# Print the results
print(f"Cosine Similarity: {cosine_sim}")
print(f"Jaccard Similarity: {jaccard_sim}")
print(f"Euclidean Distance: {euclidean_dist}")
print(f"Manhattan Distance: {manhattan_dist}")


Cosine Similarity: 0.25233420143369617
Jaccard Similarity: 0.2222222222222222
Euclidean Distance: 1.222837518696825
Manhattan Distance: 4.236033423568715


## Topic Modeling 📌 Challenge: 
Use techniques like Latent Dirichlet Allocation (LDA) to perform topic modeling on a collection of documents.

Le **Latent Dirichlet Allocation (LDA)** est un modèle statistique utilisé pour la **modélisation de sujets** dans des corpus de textes. Il permet de découvrir des structures latentes (sujets) dans un ensemble de documents sans supervision, en considérant que chaque document est une combinaison de plusieurs sujets. Voici un résumé des étapes et du concept d'**LDA** :

---

### **Concept de LDA**
LDA est un modèle de **génération de texte** qui suppose que chaque document est un mélange de plusieurs sujets, et que chaque mot dans un document est associé à un sujet spécifique. L'objectif de LDA est de **découvrir** ces sujets latents en analysant la distribution des mots dans les documents.

#### **Hypothèses principales** :
1. Chaque document est constitué d’un **mélange** de sujets (thèmes).
2. Chaque sujet est une distribution sur les mots dans le vocabulaire.
3. LDA cherche à découvrir la répartition de ces sujets et mots à partir des données observées.

---

### **Étapes du processus LDA**

1. **Initialisation** :
   - Chaque mot d'un document est attribué aléatoirement à un sujet.
   - On initialise les distributions de sujets pour chaque document et les distributions de mots pour chaque sujet.

2. **Estimation des paramètres** :
   - LDA utilise une **approche bayésienne** pour estimer les paramètres du modèle. L'idée est d'itérer sur le processus de **réaffectation** des mots à un sujet et de mettre à jour les distributions de manière à maximiser la vraisemblance des données.
   
   Les deux principales distributions sont :
   - \( \theta_d \) : La distribution des sujets pour le document \( d \).
   - \( \phi_k \) : La distribution des mots pour le sujet \( k \).

3. **Mise à jour des affectations** :
   - À chaque itération, chaque mot est réaffecté à un sujet en fonction de deux critères :
     - La probabilité que ce mot vienne du sujet, basée sur la distribution de mots de chaque sujet.
     - La probabilité que ce document soit associé à ce sujet, basée sur la distribution de sujets de ce document.

4. **Répétition jusqu'à convergence** :
   - Ce processus est répété plusieurs fois, ajustant continuellement les affectations de mots et les distributions des sujets jusqu'à ce que les distributions se stabilisent.

5. **Finalisation du modèle** :
   - Une fois l'algorithme convergé, chaque document peut être caractérisé par une distribution de sujets, et chaque sujet peut être décrit par une distribution de mots.

---

### **Résumé du processus LDA** :
1. **Suppositions** : Chaque document est un mélange de sujets et chaque sujet est une distribution sur les mots.
2. **Initialisation** : Affectation aléatoire des mots aux sujets.
3. **Itération** :
   - Mise à jour des affectations des mots aux sujets.
   - Mise à jour des distributions des sujets et des mots.
4. **Convergence** : L'algorithme converge vers un modèle stable.

---

### **Application de LDA** :
- **Découverte de sujets** dans un corpus de texte.
- **Réduction de dimension** pour des tâches comme la classification ou la recommandation.
- **Compréhension du contenu** en extrayant des thèmes ou sujets sous-jacents.

---

### **Avantages et Limites de LDA** :
- **Avantages** :
  - Identifie des sujets latents dans un corpus de textes sans supervision.
  - Utile pour l'analyse de contenu à grande échelle (extraction de thèmes).
  
- **Limites** :
  - Nécessite de spécifier le nombre de sujets à l'avance.
  - La qualité des sujets dépend fortement de la prétraitement des données et de la sélection des hyperparamètres.

LDA est un outil puissant pour explorer des corpus textuels et extraire des informations significatives sur les thèmes sous-jacents, mais il faut parfois expérimenter avec les hyperparamètres pour obtenir des résultats optimaux.

In [34]:
pip install --upgrade nltk

In [36]:
import gensim
from gensim import corpora
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Exemple de documents
documents = ["Machine learning is fascinating.", 
             "Natural language processing is a subfield of AI.",
             "Deep learning is a subset of machine learning.",
             "I love studying algorithms in computer science."]

# Prétraitement des documents
stop_words = set(stopwords.words('english'))
processed_docs = [[word for word in word_tokenize(doc.lower()) if word.isalpha() and word not in stop_words] 
                  for doc in documents]

# Créer un dictionnaire
dictionary = corpora.Dictionary(processed_docs)

# Créer le corpus
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Appliquer LDA
lda_model = gensim.models.LdaMulticore(corpus, num_topics=2, id2word=dictionary, passes=10)

# Afficher les topics
topics = lda_model.print_topics(num_words=3)
for topic in topics:
    print(topic)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tariq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tariq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\tariq/nltk_data'
    - 'c:\\Users\\tariq\\Desktop\\Quora_nlp\\.venv\\nltk_data'
    - 'c:\\Users\\tariq\\Desktop\\Quora_nlp\\.venv\\share\\nltk_data'
    - 'c:\\Users\\tariq\\Desktop\\Quora_nlp\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\tariq\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


## Word2Vec

In [40]:
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
import nltk
import string

# Ensure NLTK resources are downloaded
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except:
    print("NLTK resources download might have failed. Proceeding with alternative tokenization.")

from nltk.corpus import stopwords

# Load a pre-trained word embedding model
try:
    word2vec = api.load('glove-wiki-gigaword-100')
except Exception as e:
    print("Failed to load pre-trained model. Using a simple alternative.")
    word2vec = None

# Alternative tokenization function
def simple_tokenize(text):
    # Remove punctuation and convert to lowercase
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Split on whitespace
    tokens = text.split()
    
    # Remove stopwords
    try:
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
    except:
        # Fallback if stopwords can't be loaded
        tokens = [word for word in tokens if len(word) > 1]
    
    return tokens

# Obtenir le vecteur moyen pour un texte
def text_to_vector(text, word2vec_model):
    if word2vec_model is None:
        # Simple fallback if no pre-trained model is available
        tokens = simple_tokenize(text)
        return np.array([len(token) for token in tokens])
    
    tokens = simple_tokenize(text)
    word_vectors = [word2vec_model[word] for word in tokens if word in word2vec_model]
    
    if len(word_vectors) == 0:
        return np.zeros(word2vec_model.vector_size)  # Vecteur nul si aucun mot n'est dans le vocabulaire
    
    return np.mean(word_vectors, axis=0)

# Calculer la similarité entre deux textes
def compute_similarity(text1, text2, word2vec_model):
    vec1 = text_to_vector(text1, word2vec_model)
    vec2 = text_to_vector(text2, word2vec_model)
    
    # Handle edge cases
    if np.all(vec1 == 0) or np.all(vec2 == 0):
        return 0.0
    
    similarity = cosine_similarity([vec1], [vec2])[0][0]  # Similarité cosinus
    return similarity

# Exemple
text1 = "The cat sat on the mat."
text2 = "The cat lay on the rug."

similarity = compute_similarity(text1, text2, word2vec)
print(f"Similarité entre les deux textes : {similarity:.2f}")

Similarité entre les deux textes : 0.73


In [41]:
# Exemple
text1 = "The cat sat on the mat."
text2 = "The human is crying"

similarity = compute_similarity(text1, text2, word2vec)
print(f"Similarité entre les deux textes : {similarity:.2f}")

Similarité entre les deux textes : 0.47


## 8. **Text Generation 🖋️**
   - **Challenge**: Build a text generation model (e.g., RNNs then LSTM) that generates coherent sentences based on a given input.
   - **Solution**: This solution builds a model capable of generating new text based on patterns learned from a corpus.

In [42]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Step 1: Prepare the Data

# Load your text corpus
with open("test_corpus.txt", "r") as file:
    text = file.read().lower()

# Tokenize the text into words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create sequences of words for training the model
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad the sequences to ensure equal input lengths
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = np.expand_dims(y, axis=-1)

# Step 2: Build the Model

# Define the model architecture
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150, return_sequences=False))
model.add(Dense(total_words, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 3: Train the Model
model.fit(X, y, epochs=20, batch_size=64)

# Step 4: Generate Text

def generate_text(seed_text, model, tokenizer, max_sequence_length, num_words=50):
    for _ in range(num_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted_probs, axis=-1)[0]
        
        predicted_word = tokenizer.index_word[predicted_word_index]
        seed_text += " " + predicted_word
        
    return seed_text

# Generate new text based on a seed input
seed_text = "The future of AI"
generated_text = generate_text(seed_text, model, tokenizer, max_sequence_length)
print(generated_text)


c:\Users\tariq\Desktop\Quora_nlp\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0164 - loss: 3.9515
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.0492 - loss: 3.9442
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.1148 - loss: 3.9368
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.1311 - loss: 3.9289
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.1311 - loss: 3.9202
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.1639 - loss: 3.9102
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.1475 - loss: 3.8983
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0984 - loss: 3.8837
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0984 - loss: 3.8655
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0984 - loss: 3.8427
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.0984 - loss: 3.8144
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.0984 - loss: 3.7820
Epo